
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

     |████████████████████████████████| 175 kB 32.7 MB/s 
     |████████████████████████████████| 356 kB 59.6 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=0fe25b0fc42d713dca53c7868cc3bb33cb3211ed11f0e87cab216eb16f09682e
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491650 sha256=ac2b66a6883f9c1645b7a1d2b00e21edf91c974117cc032b4b7a909b911a7191
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built memory-profiler shap
     |████████████████████████████████| 177 kB 38.7 MB/s 


In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 168.4 MB 70 kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.6-py2.py3-none-any.whl size=168439194 sha256=ba59a673d4b03d1799c44ac772a90185b229d26eff3570fc71f61600aca58ef8
  Stored in directory: /root/.cache/pip/wheels/ee/0f/51/849ba221c4c1b11a04efb4a3427dc9cb1c4dcde218c6c98b13
Successfully built h2o


In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 33.0 MB/s 


In [5]:
!pip install BlackBoxAuditing

     |████████████████████████████████| 2.6 MB 32.4 MB/s 
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394771 sha256=46d583857f3fb09b9506a6acf2fd93807889e288c20b6e0990167895635b536a
  Stored in directory: /root/.cache/pip/wheels/05/9f/ee/541a74be4cf5dad17430e64d3276370ea7b6a834a76cb4215a
Successfully built BlackBoxAuditing


#IMPORTS

In [10]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import BlackBoxAuditing
import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
from aif360.algorithms.inprocessing import PrejudiceRemover, MetaFairClassifier, AdversarialDebiasing
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR,OptimPreproc
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os



In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp728k_1f6
  JVM stdout: /tmp/tmp728k_1f6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp728k_1f6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_p2umze
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [9]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Meta



In [13]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta0.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta0.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.8926359256710255
Accuracy 0.8884809545663148
Accuracy 0.893299678751721
Accuracy 0.8859568609453878
Accuracy 0.890546122074346
Accuracy 0.8857536132140399
Accuracy 0.8944469940339606
Accuracy 0.890546122074346
Accuracy 0.8903166590178981
Accuracy 0.8896282698485544
Accuracy 0.8921771048405598
Accuracy 0.8873336392840753
Accuracy 0.8978889398806792
Accuracy 0.887792565396971
Accuracy 0.8861863240018357
Accuracy 0.888735948612067
Accuracy 0.8845800826067003
Accuracy 0.8891693437356586
Accuracy 0.8916934373565856
Accuracy 0.8960532354290959
Accuracy 0.8969947235604496
Accuracy 0.8811381367599816
Accuracy 0.8942175309775127
Accuracy 0.8834327673244607
Accuracy 0.8942175309775127
Accuracy 0.8926359256710255
Accuracy 0.8868747131711794
Accuracy 0.8937586048646168
Accuracy 0.8866452501147315
Accuracy 0.8910050481872418
Accuracy 0.8846065611378756
Accuracy 0.8912345112436898
Accuracy 0.893299678751721
Accuracy 0.8843506195502524
Accuracy 0.8969710876548875
Accuracy 0.891030052764395

# Meta 0.2



In [14]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.2, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta2.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta2.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.8930947465014911
Accuracy 0.888022028453419
Accuracy 0.893070215695273
Accuracy 0.8852684717760441
Accuracy 0.8900871959614503
Accuracy 0.8850653819683414
Accuracy 0.8937586048646168
Accuracy 0.890546122074346
Accuracy 0.8891693437356586
Accuracy 0.8882514915098669
Accuracy 0.891947694425327
Accuracy 0.8868747131711794
Accuracy 0.8967416245984396
Accuracy 0.8871041762276274
Accuracy 0.8852684717760441
Accuracy 0.8882771277816013
Accuracy 0.8832033042680129
Accuracy 0.8889398806792106
Accuracy 0.8914639743001377
Accuracy 0.8960532354290959
Accuracy 0.8958476714842853
Accuracy 0.8799908214777421
Accuracy 0.8944469940339606
Accuracy 0.882744378155117
Accuracy 0.8937586048646168
Accuracy 0.891947694425327
Accuracy 0.8868747131711794
Accuracy 0.8935291418081689
Accuracy 0.8852684717760441
Accuracy 0.8903166590178981
Accuracy 0.8846065611378756
Accuracy 0.8907755851307939
Accuracy 0.8937586048646168
Accuracy 0.8832033042680129
Accuracy 0.8965121615419918
Accuracy 0.889424179857765

# Meta 0.4



In [15]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.4, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta4.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta4.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.8926359256710255
Accuracy 0.8882514915098669
Accuracy 0.8937586048646168
Accuracy 0.885497934832492
Accuracy 0.8900871959614503
Accuracy 0.8857536132140399
Accuracy 0.8946764570904084
Accuracy 0.890546122074346
Accuracy 0.890546122074346
Accuracy 0.8893988067921065
Accuracy 0.8921771048405598
Accuracy 0.8873336392840753
Accuracy 0.8967416245984396
Accuracy 0.8873336392840753
Accuracy 0.8861863240018357
Accuracy 0.888735948612067
Accuracy 0.8836622303809086
Accuracy 0.8891693437356586
Accuracy 0.8921523634694815
Accuracy 0.8960532354290959
Accuracy 0.8972241339756825
Accuracy 0.8815970628728774
Accuracy 0.8944469940339606
Accuracy 0.8834327673244607
Accuracy 0.8939880679210647
Accuracy 0.8924065152557926
Accuracy 0.8868747131711794
Accuracy 0.8937586048646168
Accuracy 0.8861863240018357
Accuracy 0.8903166590178981
Accuracy 0.8848359715531086
Accuracy 0.8912345112436898
Accuracy 0.8937586048646168
Accuracy 0.8838916934373566
Accuracy 0.8969710876548875
Accuracy 0.8910300527643

# Meta 0.6



In [16]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.6, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta6.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta6.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.8926359256710255
Accuracy 0.8884809545663148
Accuracy 0.17508031206975677
Accuracy 0.17989903625516293
Accuracy 0.890546122074346
Accuracy 0.8857536132140399
Accuracy 0.8944469940339606
Accuracy 0.890546122074346
Accuracy 0.3061037173015145
Accuracy 0.8896282698485544
Accuracy 0.8921771048405598
Accuracy 0.8873336392840753
Accuracy 0.8978889398806792
Accuracy 0.26548875631023405
Accuracy 0.2911886186324002
Accuracy 0.27506308786418904
Accuracy 0.23680587425424507
Accuracy 0.2452960073428178
Accuracy 0.8916934373565856
Accuracy 0.3095456631482331
Accuracy 0.8969947235604496
Accuracy 0.24483708122992198
Accuracy 0.8942175309775127
Accuracy 0.8834327673244607
Accuracy 0.2358880220284534
Accuracy 0.8926359256710255
Accuracy 0.27742083524552547
Accuracy 0.8937586048646168
Accuracy 0.26411197797154656
Accuracy 0.8910050481872418
Accuracy 0.3269098417068135
Accuracy 0.8912345112436898
Accuracy 0.893299678751721
Accuracy 0.8843506195502524
Accuracy 0.21248279027076641
Accuracy 0.891

# Meta 0.8



In [ ]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=0.8, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta8.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta8.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.8926359256710255
Accuracy 0.8884809545663148
Accuracy 0.893299678751721
Accuracy 0.8859568609453878
Accuracy 0.890546122074346
Accuracy 0.1587520073411333
Accuracy 0.8944469940339606
Accuracy 0.890546122074346
Accuracy 0.25791647544745294
Accuracy 0.22303809086737036
Accuracy 0.8921771048405598
Accuracy 0.20215695273061038
Accuracy 0.29004130335016065
Accuracy 0.2182193666819642
Accuracy 0.7888939880679211
Accuracy 0.28355127322780455
Accuracy 0.19573198715006884
Accuracy 0.8891693437356586
Accuracy 0.8916934373565856
Accuracy 0.8960532354290959
Accuracy 0.8969947235604496
Accuracy 0.19435520881138138
Accuracy 0.8942175309775127
Accuracy 0.19022487379531897
Accuracy 0.8942175309775127
Accuracy 0.8926359256710255
Accuracy 0.1642955484167049
Accuracy 0.768012849931161
Accuracy 0.8143643873336393
Accuracy 0.2319871500688389
Accuracy 0.24615737554484973
Accuracy 0.2122533272143185
Accuracy 0.893299678751721
Accuracy 0.8843506195502524
Accuracy 0.8969710876548875
Accuracy 0.89103

# Meta 1.0

In [ ]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'],axis=1)
  first_column = train.pop('first_pf')
  train.insert(0, 'first_pf', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'],axis=1)
  first_column = test.pop('first_pf')
  test.insert(0, 'first_pf', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #******************************** MetaFair Classifier regularizer*****************************
  Classifier = MetaFairClassifier(tau=1.0, sensitive_attr= 'race')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['first_pf']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta10.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Meta/Meta10.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)